<a href="https://colab.research.google.com/github/chaewonni/python-machine-learning/blob/main/%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%8B%A4%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#크롤링 실전

> 들여쓴 블록


정해진 기간 동안의 뉴스 기사를 크롤링하는 코드를 작성해봅시다.<br>
한 단어로 표시되는 뉴스 기사 수는 제한되어 있습니다. 그래서 하루 단위로 필터링하여 놓치는 기사 없이 모두 수집해보려고합니다.<br>
조금 더 재미 있는 코드들을 소개해드릴게요!

### 이번주 뉴스 기사 URL 수집
본격적으로 URL을 조작하여 지난 한 주 동안의 뉴스 기사를 수집해봅시다.<br>
여기서는 날짜 리스트를 만들어야 하는데, 어떻게 만들면 더 쉬운지 알아보고<br>
사용자 정의 함수를 구현하여 조금 더 효율적이고 이해 가능한 코드를 구현해봅시다.<br>

In [ ]:
# 필요한 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
import re
import time

In [ ]:
# 임시 URL 설정
tmp_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AI&sort=2&photo=0&field=0&pd=3&ds=2023.02.01&de=2023.02.01&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20230201to20230201,a:all&start=1'

In [ ]:
from urllib.parse import urlparse
from urllib.parse import parse_qs

# 위의 복잡한 URL은 사실 요청문으로 이루어져 있습니다. 여기에서 정보를 빠르고 쉽게 추출할 수 있어요!
params = parse_qs(urlparse(tmp_url).query)
params

{'where': ['news'],
 'sm': ['tab_pge'],
 'query': ['AI'],
 'sort': ['2'],
 'photo': ['0'],
 'field': ['0'],
 'pd': ['3'],
 'ds': ['2023.02.01'],
 'de': ['2023.02.01'],
 'mynews': ['0'],
 'office_type': ['0'],
 'office_section_code': ['0'],
 'nso': ['so:r,p:from20230201to20230201,a:all'],
 'start': ['1']}

In [ ]:
# 파라미터 변경
params['start'] = 1 # 이 값은 저희가 추후에 변경해야 하는 부분이라 리스트 형태에서 정수 형태로 미리 바꾸어두었습니다.

In [ ]:
# 날짜 리스트 생성
from datetime import datetime
from datetime import timedelta

today = datetime.today()
last_week = today - timedelta(days = 7)

today = today.strftime('%Y.%m.%d')
last_week = last_week.strftime('%Y.%m.%d')

import pandas as pd
date_list = pd.date_range(last_week, today)
#Pandas에서 날짜 리스트를 쉽게 얻을 수 있습니다. 저희가 알고 있는 문자열이 아니라, 날짜 데이터를 조금 더 유연하게 다룰 수 있는 형식으로 저장됩니다.
print(date_list)

DatetimeIndex(['2023-03-11', '2023-03-12', '2023-03-13', '2023-03-14',
               '2023-03-15', '2023-03-16', '2023-03-17', '2023-03-18'],
              dtype='datetime64[ns]', freq='D')


In [ ]:
# 뉴스 URL 수집 | 매일 첫 페이지만
url = 'https://search.naver.com/search.naver'

news_url_list = []
for date in date_list:
    date_string = date.strftime('%Y.%m.%d')
    params['ds'] = date_string
    params['de'] = date_string
    params['nso'] = 'so:r,p:from{fr}to{to}'.format(fr = date.strftime('%Y%m%d'), to = date.strftime('%Y%m%d'))
    time.sleep(1)

    res = requests.get(url, params = params)

    html = res.text
    soup = BeautifulSoup(html, 'html.parser')

    news_url_list_part = soup.find_all('a', 'info', text = '네이버뉴스')
    print(date_string, len(news_url_list_part), '개')
    news_url_list.extend(news_url_list_part)

<ipython-input-6-7c37061ce0b2>:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  news_url_list_part = soup.find_all('a', 'info', text = '네이버뉴스')


2023.03.11 4 개
2023.03.12 6 개
2023.03.13 3 개
2023.03.14 3 개
2023.03.15 4 개
2023.03.16 6 개
2023.03.17 4 개
2023.03.18 3 개


In [ ]:
def get_soup(url, params):
    res = requests.get(url, params = params)
    time.sleep(1)

    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [ ]:
def get_news_url_list(soup):
    news_url_list = soup.find_all('a', 'info', text = '네이버뉴스')
    return news_url_list

In [ ]:
def get_page_list(soup):
    page_list = list(map(lambda x: int(x.text),soup.find('div', 'sc_page_inner').find_all('a')))
    return page_list

In [ ]:
def get_daily_news(url, params):
    news_url_list = []
    n = 1

    soup = get_soup(url, params)
    page_list = get_page_list(soup)

    while n in page_list:

        soup = get_soup(url, params)
        news_url_list_part = get_news_url_list(soup)
        news_url_list.extend(news_url_list_part)

        time.sleep(1)
        page_list = get_page_list(soup)
        print(n, page_list)
        
        n += 1
        params['start'] += 10

    print(len(news_url_list), '개,', n-1, '페이지')
    return news_url_list

In [ ]:
# 뉴스 URL 수집 | 하루
date = date_list[0]
params['start'] = 1

date_string = date.strftime('%Y.%m.%d')
params['ds'] = date_string
params['de'] = date_string
params['nso'] = 'so:r,p:from{fr}to{to}'.format(fr = date.strftime('%Y%m%d'), to = date.strftime('%Y%m%d'))
time.sleep(1)

print(date_string)
daily_news_list = get_daily_news(url, params)

2023.03.11


<ipython-input-8-9bd9733922cf>:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  news_url_list = soup.find_all('a', 'info', text = '네이버뉴스')


1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
2 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
3 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
4 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
5 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
6 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
7 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
8 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
9 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
10 [6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
11 [7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
12 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
13 [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
14 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
15 [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
16 [12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
17 [12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
18 [12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
19 [12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
20 [12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
21 [12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
107 개, 21 페이지


In [ ]:
# 뉴스 URL 수집 | 모든 페이지
url = 'https://search.naver.com/search.naver'
params = parse_qs(urlparse(tmp_url).query)

news_url_list = []
for date in date_list:
    params['start'] = 1

    date_string = date.strftime('%Y.%m.%d')
    params['ds'] = date_string
    params['de'] = date_string
    params['nso'] = 'so:r,p:from{fr}to{to}'.format(fr = date.strftime('%Y%m%d'), to = date.strftime('%Y%m%d'))
    time.sleep(1)
    
    print(date_string)
    daily_news_list = get_daily_news(url, params)
    news_url_list.extend(daily_news_list)

# 1주일 분량에 46분 30초 소요

2023.03.11


<ipython-input-8-9bd9733922cf>:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  news_url_list = soup.find_all('a', 'info', text = '네이버뉴스')


1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
2 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
3 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
4 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
5 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
6 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
7 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
8 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [ ]:
# 수집된 URL 개수 확인
len(news_url_list)

0

In [ ]:
# 뉴스 URL 추출
news_url_list = list(map(lambda x : x.get('href'), news_url_list))

# for문을 이용하지 않고 뉴스 맵 함수를 이용하여 빠르게 URL을 추출합니다.

In [ ]:
# URL 데이터 프레임으로 변환
df = pd.DataFrame(news_url_list, columns = ['URL']); df

,URL


In [ ]:
# URL 데이터 프레임 저장
# Google Drive 불러온 후

path = '/content/drive/MyDrive/[deep daiv.] 머신러닝 입문/'
file_name = '네이버 뉴스_AI_{}.csv'.format(date_string)
df.to_csv(path + file_name, index = False)

OSError: ignored

## 뉴스 기사 본문 수집 복습

In [ ]:
from urllib.request import urlopen

In [ ]:
df = pd.read_csv(path + '네이버 뉴스_AI_{}.csv'.format(date_string))

FileNotFoundError: ignored

In [ ]:
test_url = df['URL'][1]

res = urlopen(test_url)
html = res.read().decode('utf-8')
soup = BeautifulSoup(html, 'lxml')

IndexError: ignored

In [ ]:
# 뉴스 기사 제목 수집
title = soup.find('h2').text
print(title)

In [ ]:
# 뉴스 기사 본문 수집
content = soup.find('div', {'id' : 'dic_area'}).text.strip()

content = re.sub('\n+', '\n', content) # 이 코드는 정규표현식을 활용하여 불필요한(\n) 띄어쓰기를 줄이는 코드입니다.
print(content)

In [ ]:
# 언론사 수집
press = soup.find('img', 'media_end_head_top_logo_img light_type').get('title')
print(press)

In [ ]:
# 날짜 데이터 변환
from datetime import datetime, timedelta, timezone

date = soup.find('span','media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').get('data-date-time')
date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
print(date)

In [ ]:
# 반복문의 진행 상황을 알려주는 패키지: tqdm
from tqdm.auto import tqdm

In [ ]:
data_list = []
error_list = []

for url in tqdm(df['URL']):
    try:
        res = urlopen(url)
        time.sleep(1)
    except:
        error_list.append(url)
        print(url)
        time.sleep(1)
        continue

    if 'entertain.naver.com' in res.url or 'sports.news.naver.com' in res.url:
        error_list.append(url)
        print(url)
        continue

    html = res.read().decode('utf-8')
    soup = BeautifulSoup(html, 'lxml')

    # 타이틀
    title = soup.find('h2').text.strip()

    # 본문 내용
    content = soup.find('div', {'id' : 'dic_area'}).text.strip()
    content = re.sub('\n+', '\n', content)

    # 언론사
    press = soup.find('img', 'media_end_head_top_logo_img light_type').get('title')

    # 날짜
    date = soup.find('span','media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').get('data-date-time')
    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

    data = title, content, press, date
    data_list.append(data)

    if len(data_list) % 100 == 0:
        news = pd.DataFrame(data_list, columns = ['Title', 'Content', 'Press', 'Date'])
        news.to_csv(path + '네이버 뉴스 본문_AI_11월 1주차.csv', index = False)
        
        with open(path + 'news_list_11월 1주차.p', 'wb') as f:
            pickle.dump(data_list, f)

        datetime_utc = datetime.utcnow()
        timezone_kst = timezone(timedelta(hours = 9))
        datetime_kst = datetime_utc.astimezone(timezone_kst)
        print(datetime_kst)

In [ ]:
len(data_list)

In [ ]:
data_list[-1]

In [ ]:
news = pd.DataFrame(data_list, columns = ['Title', 'Content', 'Press', 'Date'])
news.to_csv(path + '네이버 뉴스 본문_AI.csv', index = False)

import pickle
with open(path + 'news_list_{}.p'.format(date_string), 'wb') as f:
    pickle.dump(data_list, f)

In [ ]:
with open(path+'news_list_{}.p'.format(date_string), 'rb') as f:
    data_list = pickle.load(f)